In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

In [2]:
from architectures.lstm import SimpleLSTM
from architectures.gpt import GPTDecoder

vocab_size = 50_000   # number of tokens
embed_dim = 384     # embedding dimension
hidden_dim = 384     # LSTM hidden size
num_layers = 2

lstm = SimpleLSTM(vocab_size, embed_dim, hidden_dim, num_layers)
param_count = sum(p.numel() for p in lstm.parameters() if p.requires_grad)
print(f"LSTM has {param_count} params")

LSTM has 21615440 params


In [3]:
vocab_size = 50_000
embed_dim = 256
num_heads = 8
ff_hidden_dim = 2048
num_layers = 6
context_length = 128
dropout = 0.1

gpt = GPTDecoder(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    num_heads=num_heads,
    ff_hidden_dim=ff_hidden_dim,
    num_layers=num_layers,
    context_length=context_length,
    dropout=dropout
)

param_count = sum(p.numel() for p in gpt.parameters())
print(f"GPT has {param_count} params")

GPT has 20690944 params


In [4]:
import torch

def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

In [5]:
import time
from functools import wraps

def measure_inference_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        elapsed = end - start
        print(f"[{func.__name__}] Inference time: {elapsed:.4f} seconds")
        return result
    return wrapper

In [6]:
from torch.functional import F

@measure_inference_time
@torch.no_grad()
def generate_text_lstm(model, tokenizer, prompt, max_new_tokens=20, temperature: float = 1.0, device=None):
    if device is None:
        device = choose_device()
    
    model.to(device)
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

    generated_tokens = tokens.copy()

    with torch.no_grad():
        hidden = None
        for _ in range(max_new_tokens):
            out, hidden = model(input_ids, hidden)

            logits = out[0, -1, :]
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)

            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
            input_ids = torch.tensor([[next_token_id]], dtype=torch.long).to(device)

            generated_tokens.append(next_token_id.item())
    
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

@measure_inference_time
@torch.no_grad()
def generate_text_gpt(model, tokenizer, prompt, max_new_tokens=20,  temperature=1.0, device=None):
    if device is None:
        device = choose_device()

    model.to(device)
    model.eval()

    tokens = tokenizer.encode(prompt, add_special_tokens=False)
    input_ids = torch.tensor(tokens, dtype=torch.long, device=device).unsqueeze(0)

    for _ in range(max_new_tokens):
        logits = model(input_ids)

        logits = logits / temperature
        logits = logits[0, -1, :]
        
        probs = F.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1).reshape(1, 1)
        input_ids = torch.cat([input_ids, next_token_id], dim=1)

    generated_tokens = input_ids[0].tolist()
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return text

In [7]:
lstm.load_state_dict(torch.load("saved_models/lstm_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [8]:
gpt.load_state_dict(torch.load("saved_models/gpt_final.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
prompts = [
    "Czasem jedno słowo potrafi zmienić cały dzień.",
    "Wczoraj ktoś zostawił mi kartkę na ławce, bez podpisu.",
    "No dobra, ale kto w ogóle uznał, że to ma sens?",
    "To miało być tylko na chwilę, a wyszło jak zawsze.",
    "Nie wiem, czy to przez pogodę, czy przez ludzi, ale dziś wszystko wydaje się dziwnie ciche.",
    "„Nie klikaj tam” — powiedział, zanim ekran zgasł.",
    "W sumie nie planowałem o tym mówić, ale skoro już tu jesteś…",
    "Dwa dni bez snu i nagle wszystko zaczyna się układać. Ironia, co?",
    "Kiedy byłem mały, myślałem, że dorośli wszystko wiedzą.",
    "Czasami po prostu trzeba usiąść, włączyć coś spokojnego i udawać, że świat się nie pali."
]

In [10]:
import textwrap

for prompt in prompts:
    text = generate_text_gpt(gpt, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
    print(textwrap.fill(text, width=80), "\n")

[generate_text_gpt] Inference time: 1.2819 seconds
Czasem jedno słowo potrafi zmienić cały dzień . Arka Rzeka – akt nadający tytuł
podczas w Polsce z 2 lutego 2021 na Międzynarodowym Festiwalu Filmowym w
Niemczech . Promujący tematycznie ukazywał się po 2010 roku . Był on tylko
miejscem ponad jubileuszu 100 lat . Wykonawcą był do 2000 Olgierd Mens , a
dziekanem tejże książki jest Jarosław Lubicz . Następnego dnia droga ( trasa
impreza na polskich kat . Q ) jest obiektem trudności w wydobyciu kluczowych
technologii komputerowych oraz udostępnień Enigma . W 1997 roku siły MONNP
zostały 

[generate_text_gpt] Inference time: 1.1858 seconds
Wczoraj ktoś zostawił mi kartkę na ławce , bez podpisu . Na ich miejscu ppłk
Bronisław Bota pobił rekord brunatny . P . C . opowiadał swoje spotkanie 31 :
GKS Tychy 5 : 2 z TKH Toruń 3 : 1 do GKS Tychy 2 : 1 i GKS Tychy 2 : 3 , jednak
Cracovia z TKH Toruń po raz drugi w Oświęcimiu miała zadecydować o jego życiu 3
: 1 . W rewanżu oglądało 2 tormę w bramki

In [11]:
for prompt in prompts:
    text = generate_text_lstm(lstm, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
    print(textwrap.fill(text, width=80), "\n")

[generate_text_lstm] Inference time: 0.2231 seconds
Czasem jedno słowo potrafi zmienić cały dzień . Trzy święta obrzeżne są odmienne
od małpy . Obecna forma środowiska płynących wynosi 1 , 722 – 100 km i duże
znaczenie jako słaba wielkość donora , np . : “ fastyk ” , którego języku
sprowadzano Indian z obu stron . PropaBudowlanych mistrza z systemu Los Angeles
, głównie z Lindą na czele . Kowal . Poza tym w dziedzinie deklaracje ufności
pogorszyło sytuacji – przede wszystkim uprzedzeń lub eliksirów oraz zmiany
sprzeczności w znacznie skrótach poruszających równoległymi kamer odbytowanych 

[generate_text_lstm] Inference time: 0.2064 seconds
Wczoraj ktoś zostawił mi kartkę na ławce , bez podpisu . Kadrot używa najdłużej
, $ dowódca , gdy oboje będą rozpoczynającym się gole . To przyjemność w pełni
sprawnej na koniec sezonu . Drużyna musi wejść w radykalni Trójcy w Wardin . Po
zrzuceniu , zawodnik , związała się z tym , że odwrócono jeszcze pas mistrzowski
w historii futbolu , który z ko

In [20]:
prompt = "W Oceanie Atltantyckim na dnie żyją sobie "
text = generate_text_lstm(lstm, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
print(textwrap.fill(text, width=80), "\n")

[generate_text_lstm] Inference time: 0.2241 seconds
W Oceanie Atltantyckim na dnie żyją sobie zarówno odmiana antykomuniorstwo , jak
i frakcja wewnętrznej społeczeństwa , a nawet ich aktywistów . Organizowano
również trudności z zgromadzeniem i zastraszeniami . Efekty rozbronia . Obie
struktury nie były pierwszymi jednostkami ratowniczymi niezawodność służby
zdrowia do I wojny światowej . Na obowiązujących zasad piśmienniczych korporacja
sytuacji radzieckiej jednostka została ostatecznie ostrzelicana , odsunięta
przez Polskę , podobnie jak na konkretne jednostki , w związku z tym w naprawą
elektrycznym wybuchł samodzielne rządy społeczne . Nazwy Podziemno - 



In [17]:
text = generate_text_lstm(lstm, tokenizer, prompt, max_new_tokens=100, temperature=1.0, device="cpu")
print(textwrap.fill(text, width=80), "\n")

[generate_text_lstm] Inference time: 0.2203 seconds
Był to rok 1410 , gdy wojska polskie i krzyżackie starły się ze sobą pod
Kodomrwem , toteż cały czas ważniejszy od miast w Czechach . Rabin Herzl
ponownie przypuszczalnie dla tego miasta płacili jak 12 włoskich srebrnych .
Naprzeciw cysternicy z 1411 r . , bo pomimo niemożności Bożego pnia nie została
wykonana jednostka pod zamek w 1461 roku , odrzucił jego imię " Arn " ( 1731 ) .
Uzyskał godność hrabiego Marian Karczmize ze Stomitry Donalda von Seszele . 7
czerwca 1185 roku poeta rezygnował 

